In [41]:
from openai import OpenAI
openai_client = OpenAI()
def get_openai_response(text,src_lang='en', tgt_lang='he'):
    completion = openai_client.chat.completions.create(
        model="o4-mini-2025-04-16",
        messages=[
            {
                'role': 'assistant',
                'content': f"""You are a professional translator. Your task is to translate the text from {src_lang} to {tgt_lang}. 
                'You must return the translation without any additional text. 
                'You must not return the original text. 
                the document are an engineering document,specific to the field manufacture engendering on the medical devices industry. 
                """
            },
            {
                "role": "user",
                "content": f"Translate this text to Hebrew: {text} just return the translation without any additional text."
            }
        ]
    )

    return completion.choices[0].message.content

In [82]:
import os, json

import unstructured_client
from unstructured_client.models import operations, shared

client = unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)
def get_list_of_elements_from_file_with_unstructured_client(file_path:str,src_lang='he', tgt_lang='en'):
    """
    This function uses the Unstructured Client to process a file and return a list of elements.
    :param filename: The path to the input file.
    :param src_lang: Source language for translation.
    :param tgt_lang: Target language for translation.
    :return: List of elements processed from the file.
    """
    

    import unstructured_client
    from unstructured_client.models import operations, shared

    client = unstructured_client.UnstructuredClient(
        api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
    )

    filename = file_path

    req = operations.PartitionRequest(
        partition_parameters=shared.PartitionParameters(
            files=shared.Files(
                content=open(filename, "rb"),
                file_name=filename,
            ),
            strategy=shared.Strategy.HI_RES,
            # vlm_model="gpt-4o",
            # vlm_model_provider="openai",
            # languages=['eng'],
            # split_pdf_page=True,            # If True, splits the PDF file into smaller chunks of pages.
            # split_pdf_allow_failed=True,    # If True, the partitioning continues even if some pages fail.
            # split_pdf_concurrency_level=15  # Set the number of concurrent request to the maximum value: 15.
        ),
    )

    try:
        res = client.general.partition(
            request=req
        )
        element_dicts = [element for element in res.elements]
        return element_dicts
        # Print the processed data's first element only.
        # print(element_dicts[0])

        # Write the processed data to a local file.
        # json_elements = json.dumps(element_dicts, indent=2)

        # with open("PATH_TO_OUTPUT_FILE", "w") as file:
        #     file.write(json_elements)
    except Exception as e:
        print(e)

In [85]:
response= get_list_of_elements_from_file_with_unstructured_client("החלפת קם תחנות בדיקה.docx")

INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"


In [87]:
for el in response:
    if el['type']!= 'Image':
        print(el)

{'type': 'NarrativeText', 'element_id': 'aacb8d6078f4f69252e06efdfe043cda', 'text': 'החלפת קאם (532550) המניע ממרכז המכונה את תחנות התמיכה והבדיקה במכונה 222', 'metadata': {'category_depth': 0, 'emphasized_text_contents': ['החלפת ק', 'א', 'ם (532550) המניע', 'ממרכז המכונה', 'את תחנות', 'התמיכה ו', 'הבדיקה במכונה 222'], 'emphasized_text_tags': ['b', 'b', 'b', 'b', 'b', 'b', 'b'], 'page_number': 1, 'languages': ['heb'], 'filename': 'החלפת קם תחנות בדיקה.docx', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'}}
{'type': 'UncategorizedText', 'element_id': 'c8cf54a405a20ba89e5c5015033beaba', 'text': 'רשם: צביקה איבגי ופאדי סלימאן. 12.12.2024', 'metadata': {'category_depth': 0, 'page_number': 1, 'languages': ['heb'], 'filename': 'החלפת קם תחנות בדיקה.docx', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'}}
{'type': 'UncategorizedText', 'element_id': '05eb9af97472376b97cdde5ea35c0ddf', 'text': 'צוות טכני מבצע: אוסטין א

In [47]:
translated_dict = []
for i in range(0,10):
    print(response[i])
    translated_text = get_openai_response(response[i]['text'],'he','en')
    translated_dict.append({"type": response[i]['type'], "text": translated_text})
    

{'type': 'NarrativeText', 'element_id': 'aacb8d6078f4f69252e06efdfe043cda', 'text': 'החלפת קאם (532550) המניע ממרכז המכונה את תחנות התמיכה והבדיקה במכונה 222', 'metadata': {'category_depth': 0, 'emphasized_text_contents': ['החלפת ק', 'א', 'ם (532550) המניע', 'ממרכז המכונה', 'את תחנות', 'התמיכה ו', 'הבדיקה במכונה 222'], 'emphasized_text_tags': ['b', 'b', 'b', 'b', 'b', 'b', 'b'], 'page_number': 1, 'languages': ['heb'], 'filename': 'החלפת קם תחנות בדיקה.docx', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'}}


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'type': 'UncategorizedText', 'element_id': 'c8cf54a405a20ba89e5c5015033beaba', 'text': 'רשם: צביקה איבגי ופאדי סלימאן. 12.12.2024', 'metadata': {'category_depth': 0, 'page_number': 1, 'languages': ['heb'], 'filename': 'החלפת קם תחנות בדיקה.docx', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'}}


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'type': 'UncategorizedText', 'element_id': '05eb9af97472376b97cdde5ea35c0ddf', 'text': 'צוות טכני מבצע: אוסטין אשמוז, גסאן מארון.', 'metadata': {'category_depth': 0, 'page_number': 1, 'languages': ['heb'], 'filename': 'החלפת קם תחנות בדיקה.docx', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'}}


...

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'type': 'ListItem', 'element_id': '3c17097b39c108fc1bab18c6e244cba9', 'text': 'כשהתחנה במצב נעיצה הבורג (המעצור המכני) נגע בבלוק שמחזיק אותו - הושחז הבלוק.', 'metadata': {'category_depth': 1, 'page_number': 2, 'languages': ['heb'], 'filename': 'החלפת קם תחנות בדיקה.docx', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'parent_id': 'daa345baa72261756a6a2ed3aff365e9'}}


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'type': 'ListItem', 'element_id': 'a00c264cac8fed795a8cd8acd867d265', 'text': 'הבורג העליון שמחזיק את כל המכלול של המעצור המכני נשבר- הוחלף בחדש.', 'metadata': {'category_depth': 1, 'page_number': 2, 'languages': ['heb'], 'filename': 'החלפת קם תחנות בדיקה.docx', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'parent_id': 'daa345baa72261756a6a2ed3aff365e9'}}


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [48]:
translated_dict

[{'type': 'NarrativeText',
  'text': 'Replacement of cam (532550) that drives the support and testing stations from the machine center on machine 222'},
 {'type': 'UncategorizedText',
  'text': 'Registrar: Tzvika Ivgi and Fadi Slimiman. 12.12.2024'},
 {'type': 'UncategorizedText',
  'text': 'Technical operations team: Austin Ashmoz, Ghassan Maroun.'},
 {'type': 'NarrativeText',
  'text': 'Background: Over the past year the same cam has been replaced twice (before 11/2024), and the spring piston was replaced once (for extra safety). Each time, after almost two months, the cam wore out again.'},
 {'type': 'ListItem',
  'text': 'On 12.11.2024, in work order No. 581433, lot 242668277-03, bag number 291, the machine was inspected again and the findings are as follows:'},
 {'type': 'ListItem',
  'text': 'The cam and the cam follower were worn out – replaced with new ones.'},
 {'type': 'ListItem',
  'text': 'Station 24 – when the station descends to its lowest position, it stops on the block,